In [ ]:
# initial setup
try:
    # settings colab:
    import google.colab
    
except ModuleNotFoundError:    
    # settings local:
    %run "../../../common/0_notebooks_base_setup.py"

<img src="../../../common/logo_DH.svg" align='left' width=50%/>

# Regresión Logística - Práctica Independiente
## Tabla de Contenidos

- [1. Introducción](#intro)
- [2. Predicción de enfermedades cardíacas](#pred)
    - [2.1. Descripción del dataset](#descripcion)
    - [2.2. EDA - Exploratory data analysis](#eda)
    - [2.3. Clasificación de pacientes](#clf)
        - [2.3.1. Preparación de los datos](#prep)
        - [2.3.2. Ajuste del modelo](#fit)
        - [2.3.3. Evaluación del modelo](#eval)
        - [2.3.4. Optimización del modelo](#opt)       


<a id="intro"></a>
## Introducción
En 1948 en Framingham, un pequeño pueblo en Massachusetts, Estados Unidos, comenzó uno de los estudios médicos más famosos de la historia y que aun continua vigente. El estudio, conocido también como Framingham Heart Study, consistió en la participación voluntaria de 3 generaciones de habitantes del pueblo, quienes fueron sometidos a estudios médicos regulares cada 3 a 5 años, generando un gran volumen de datos sobre la salud mental y física, especialmente de enfermedades cardiovasculares de todos los involucrados.<br>
Los aportes de este estudio son numerosos, entre los cuales se destacan las relaciones entre el riesgo de tener una enfermedad cardíaca con varios aspectos de la vida cotidiana y del estado de salud general de las personas, tales como la presión sanguínea, los niveles de colesterol, el consumo de cigarrillos y factores psico-sociales. El término "factor de riesgo" fue acuñado como parte de los hallazgos de este estudio. 
Para más información sobre el estudio se puede ingresar a su sitio web https://framinghamheartstudy.org/.
El dataset puede ser descargado de https://www.kaggle.com/amanajmera1/framingham-heart-study-dataset

<a id="pred"></a>
## Predicción de enfermedades cardíacas
En esta práctica trabajaremos con un extracto del dataset resultante del estudio, en el que se cuenta con información médica de varios participantes, entre la que se encuentra el riesgo de que esa persona tenga una enfermedad coronaria durante los próximos 10 años. Nos propondremos entrenar un regresor logístico que permita clasificar y identificar los pacientes en riesgo de tener este tipo de enfermedad basándonos en sus datos clínicos.

<a id="descripcion"></a>
### Descripción del dataset
El dataset cuenta con las siguientes variables:
- age: Edad.
- male: Género.
- education: 1 = Some High School; 2 = High School or GED; 3 = Some College or Vocational School; 4 = college
- currentSmoker: Si la persona fuma o no
- cigsPerDay: la cantidad de cigarrillos que la persona fuma por día en promedio.
- BPMeds: si la persona consume medicación para la presión sanguínea.
- prevalentStroke: Si la persona tuvo un infarto anteriormente.
- prevalentHyp: si la persona tiene hipertensión.
- diabetes: si la persona tuvo diabetes.
- totChol: nivel de colesterol total.
- sysBP: presión sanguínea sistólica.
- diaBP:: presión sanguínea diastólica.
- BMI: índice de masa corporal.
- heartRate: frecuencia cardíaca.
- glucose: nivel de glucosa en sangre.
- TenYearCHD: si la persona está en riesgo de tener una enfermedad coronaria dentro de los próximos 10 años.

Para obtener más información sobre las variables, buscar en https://www.kaggle.com/amanajmera1/framingham-heart-study-dataset

<a id="eda"></a>
### EDA - Exploratory data analysis
Importar el dataset y realizar un análisis exploratorio del mismo. 
- ¿Cuántos registros hay?
- ¿Qué tipo de variable es cada una?
- ¿Hay valores faltantes?
- ¿Hay valores fuera del rango esperado?
- ¿En qué tipo de dato están almacenados?
- ¿Todas las variables son médicas o hay otra información en el dataset?
- ¿Hay correlación entre las variables?
- ¿En qué rango está cada una?
- Cómo son las proporciones de las variables categóricas?

Realizar las visualizaciones adecuadas para responder estas preguntas

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('../Data/framingham.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.isna().sum()

In [ ]:
df.isna().any(axis=1).sum()

Hay 4240 registros de los cuales 582 tienen valores faltantes

In [ ]:
582/4240*100

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=5, figsize=(18,12))
fig.suptitle('Histogramas normalizados')
for c, ax in zip(df.columns[:-1], axes.flatten()):
    sns.distplot(df.loc[df['TenYearCHD']==0, c].dropna(), norm_hist=True, kde=False, ax=ax)
    sns.distplot(df.loc[df['TenYearCHD']==1, c].dropna(), norm_hist=True, kde=False, ax=ax)
    ax.legend(['TenYearCHD = 0', 'TenYearCHD = 1'])

In [ ]:
plt.figure(figsize=(8,12))
sns.heatmap(df.corr()[['TenYearCHD']], annot=True)

<a id="clf"></a>
### Clasificación de pacientes
Ahora utilizaremos una regresión logística como clasificador de personas y el objetivo será etiquetar a aquellas que tengan riesgo de padecer una enfermedad coronaria en los próximos 10 años.
<a id="prep"></a>
#### Preparación de los datos
¿Qué debemos hacer con el dataset antes de entrenar el modelo?

In [ ]:
# Eliminamos los registros con valores faltantes ya que son sólo un 13% del dataset
df.dropna(inplace=True)

In [ ]:
# Preparamos la matriz de features y el vector objetivo
X = df.drop(columns=['TenYearCHD'])
y = df['TenYearCHD']

In [ ]:
# Separamos un set de testeo para evaluar el modelo
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 0)

In [ ]:
# Verificamos que coincidan las proporciones del target
y_train.mean(), y_test.mean()

In [ ]:
# Estandarizamos los datos para que la regularización sea correcta
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

<a id="fit"></a>
#### Ajuste del modelo

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train_std, y_train)

<a id="eval"></a>
#### Evaluación del modelo
Con el modelo entrenado podemos comenzar a evaluar su performance y ver si podemos hacer algo para mejorarlo

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
accuracy_score(y_train, lr.predict(X_train_std))

In [ ]:
sns.heatmap(confusion_matrix(y_train, lr.predict(X_train_std)), annot=True, fmt='4d')
plt.xlabel('Predichos')
plt.ylabel('Reales')

Observamos que hay una accuracy del 85%, pero que de los 418 casos positivos, sólo 32 (7.6%) están siendo detectados. Esto implica que nuestro clasificador le está diciendo a mucha gente que está fuera de peligro cuando en realidad no es así. Intentemos modificar el modelo para mejorar la predicción en estos casos

<a id="opt"></a>
#### Optimización del modelo

In [ ]:
lr.predict_proba(X_train_std[y_train==0])[:,1]

In [ ]:
# Histograma de probabilidades
sns.distplot(lr.predict_proba(X_train_std[y_train==0])[:,1])
sns.distplot(lr.predict_proba(X_train_std[y_train==1])[:,1])
ylim = plt.ylim()
plt.vlines(0.5, ylim[0], ylim[1])
plt.ylim(ylim)
plt.legend(['Umbral', 'TenYearCHD = 0', 'TenYearCHD = 1'])

Dada la naturaleza del probelma, conviene utilizar una estrategia que impida clasificar como pacientes sanos a aquellos que están en riesgo.

In [ ]:
umbral = 0.2
probs = lr.predict_proba(X_train_std)[:,1]
y_pred_train = probs > umbral

In [ ]:
accuracy_score(y_train, y_pred_train)

In [ ]:
sns.heatmap(confusion_matrix(y_train, y_pred_train), annot=True, fmt='4d')
plt.xlabel('Predichos')
plt.ylabel('Reales')

Modificando el umbral de decisión, baja considerablemente la exactitud del modelo, pero ahora detecta correctamente 225 (53.8%) de los casos positivos 